In [ ]:
!pip freeze > requirements.txt

In [1]:
import spacy
import os
import pandas as pd
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
from collections import defaultdict
import ast
from tqdm import tqdm

###### Constants

In [ ]:
window = 15

In [ ]:
data_directory = os.path.join(os.getcwd(), "data")
corpus_fpath = os.path.join(data_directory, "corpus.txt")
out_file = os.path.join(data_directory, "noun_context_corpus.txt")

In [ ]:
with open(corpus_fpath, "r") as f:
    corpus_data = f.readlines()


In [ ]:
with open(out_file, "w") as f:
    for txt_line in corpus_data:
        nlp_data = nlp(txt_line.replace("\n", ""))
        max_tokens = len(nlp_data)
        for i, tok in enumerate(nlp_data):
            if "NN" in tok.tag_:
                start = max(0, i-window)
                end = min(i+window, max_tokens)
                left_context = [t.text for t in nlp_data[start:i]]
                right_context = [t.text for t in nlp_data[i:end]]
                noun_in_context = " ".join(left_context + right_context)
                f.write(f"{tok.text} - {noun_in_context}\n")

In [ ]:
nlp_data[2]

In [ ]:
def prepare_corpus_data_with_context(corpus_filepath, window=5, force_update=False):
    nlp = spacy.load('en')
    with open(corpus_fpath, "r") as f:
        corpus_data = f.readlines()
    out_file = os.path.join(os.path.dirname(corpus_fpath), f"window_{window}_context_corpus.txt")
    contextual_data = []
    if os.path.exists(out_file) and (not force_update):
        print("Found preexisting file. Loading context data from file")
        with open(out_file, "r") as f:
            contextual_data = f.readlines()
    else:
        print("No pre created corpus found, or force update flag is true. Generating contextual data from corpus")
        with open(out_file, "w") as f:
            for txt_line in corpus_data:
                nlp_data = nlp(txt_line.replace("\n", ""))
                max_tokens = len(nlp_data)
                for i, tok in enumerate(nlp_data):
                    if "NN" in tok.tag_:
                        start = max(0, i-window)
                        end = min(i+window, max_tokens)
                        left_context = [t.text for t in nlp_data[start:i]] + [f"<{tok.text}>"]
                        right_context = [t.text for t in nlp_data[i+1:end]]
                        noun_in_context = f"<{tok.text}> - {' '.join(left_context + right_context)}"
                        contextual_data.append(noun_in_context)
                        f.write(noun_in_context + "\n")
    return contextual_data


In [ ]:
contextual_data = prepare_corpus_data_with_context(corpus_fpath, window, force_update=True)

In [ ]:
contextual_data

In [ ]:
def get_sense_group_from_corpus(context_data, wsd_model, sense_vectors, output_file):
    output = {
        "word": [],
        "context": [],
        "sense_id": [],
        "sense_group_name": [],
        "sense_group_num": [],
        "sense_probability": [],
        "related_senses": []
    }
    
    for row in context_data:
        word, ctx = row.split(' - ')
        sense_id, sense_probs = wsd_model.disambiguate(corpus_data, test_word)
        sense_probability = max(sense_probs)
        sense_group_name, sense_group_num = sense_id.split("#")
        related_senses = [r_senseid for r_senseid,_ in sense_vectors.wv.most_similar(sense_id)]
        related_senses_l2 = [r_senseid for r_senseid,_ in sense_vectors.wv.most_similar(related_sense) for related_sense in related_senses]
        output["word"].append(word)
        output["context"].append(ctx)
        output["sense_id"].append(sense_id)
        output["sense_group_name"].append(sense_group_name)
        output["sense_group_num"].append(sense_group_num)
        output["sense_probability"].append(sense_probability)
        output["related_senses"].append(related_senses+related_senses_l2)
    
    output_df = pd.DataFrame(output)
    output_df.to_csv(output_file, index=False)
    
    return output_df

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

In [ ]:
nlp = spacy.load("en")
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

In [ ]:
token = nlp('prices')[0]

In [ ]:
token._.wordnet.synsets()[0].hyponyms()

In [ ]:
token._.wordnet.synsets()[0].part_meronyms()  

#### HHM Algo - 

###### For hypernym extraction
1. Create empty dict of hypernyms  -   
    ```
    {
	"word_num": [{
		"hypernym_1": {
			"rev_map": {
				"direct": [{
					"sense_id": synset_object
				}],
				"L1": [{}],
				"L2": [{}]
			},
			"weight": num
		},
		"hypernym_2": {}
	}]}
    ```
- For each word in data 
    - extract hypernym from synset mappings for word#number - Highest weight(8)
    - extract hypernym for all other synset mappings(5)
    - For each related word level 1 
        - Extract hypernym for word#number - weight(3)
        - Extract hypernym for all others - weight(2)
    - For each related word level 2
        - Extract hypernym for word#number - weight(1)
        - Extract hypernym for all others - weight(0.5)
    - Keep adding the hypernym weight to the right key. 
    - Identify the right hypernym - One with highest weight and present in data, else next highest, and so on.
    - maintain a reverse map for each hypernym(for hyponymy calculation)

##### For hyponymy
1. Initialize dict for hyponyms - ```{word: [hyponym1, hyponym2, ...]}```
- Use rev map created in previous step to identify hyponyms of the noun under consideration.
- For each entry in rev_map, get hyponyms from the synset object. Maintain the direct, L1, L2 structure.
- Extract and preserve hyponyms that exist in the data

##### For meronymy
1. Use hypernymy-hyponymy relation to create a tree structure. 
2. For each level of the tree, identify meronyms for each node, and preserve common meronyms, that are also present in the data.

#### Hypernymy extractor

In [ ]:
weights = {
    "direct_match": 8,
    "direct_nomatch": 5,
    "l1_match": 3,
    "l1_nomatch": 2,
    "l2_match": 1,
    "l2_nomatch": 0.5,
}

In [ ]:
import spacy
import os
import pandas as pd
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

nlp = spacy.load("en")
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')


df = pd.read_csv("corpus_sense_mapping.csv")
df.head(n=20)

In [ ]:
def generate_hypernymy(word, idx, hypernymy_dict, level="l1"):
    token = nlp(str(word))[0]
    synsets = token._.wordnet.synsets()
    for syn in synsets:
        if syn.name().split(".n.")[0] == word and int(syn.name().split(".n.")[1]) == idx:
            match_str = "match"
        else:
            match_str = "nomatch"

        hypernym_syn = syn.hypernyms()
        hyponym_syn = syn.hyponyms()
        meronym_syn = syn.part_meronyms()
        if len(hypernym_syn) >0:
            hypernym = hypernym_syn[0].name().split('.')[0]
            rev_map = {
                f"{word}#{idx}":syn,
                "hyponyms": [hyp.name().split(".")[0] for hyp in hyponym_syn],
                "meronyms": [mero.name().split(".")[0] for mero in meronym_syn]
            }
            if level in hypernymy_dict[hypernym]["rev_map"]:
                hypernymy_dict[hypernym]["rev_map"][level].append(rev_map) 
            else:
                hypernymy_dict[hypernym]["rev_map"][level] = [rev_map]

            if "sum_weight" in hypernymy_dict[hypernym]:
                hypernymy_dict[hypernym]["sum_weight"] += weights[f"{level}_{match_str}"]
            else:
                hypernymy_dict[hypernym]["sum_weight"] = weights[f"{level}_{match_str}"]

In [ ]:
def extract_hyponyms_meronym(hypernymy_dict, hypernym, kind="hyponyms"):
    nym = []
    reverse_map = hypernymy_dict[hypernym]['rev_map']
    extract_from = []
    if "direct" in reverse_map:
        extract_from.append("direct")
    if "l1" in reverse_map:
        extract_from.append("l1")
    
    for relation in extract_from:
        num_relations = len(reverse_map[relation])
        for i in range(num_relations):
            nym += reverse_map[relation][i][kind]
            
    return list(set(nym))
            

In [ ]:
all_hypernymy = []
all_hhm_map = []
all_hyponyms = []
all_meronyms = []
num_words = len(df)
for _, row in tqdm(df[["sense_group_name", "sense_group_num", "related_senses"]].iterrows(), total=num_words):
    word, idx, r_senses = row
    
    hypernymy_dict = defaultdict(lambda: defaultdict(dict))
    generate_hypernymy(word, idx, hypernymy_dict, level="direct")
    r_senses_list = ast.literal_eval(r_senses)
    for w in r_senses_list:
        related_word, related_idx = str(w).split("#")
        generate_hypernymy(related_word, related_idx, hypernymy_dict)
    if len(hypernymy_dict)>0:
        hypernymy_dict_sorted = {k: v for k, v in sorted(hypernymy_dict.items(), key=lambda item: item[1]['sum_weight'], reverse=True)}
        hypernym_name = list(hypernymy_dict_sorted.keys())[0]
        
        hyponyms = extract_hyponyms_meronym(hypernymy_dict_sorted, hypernym_name, kind="hyponyms")
        meronyms = extract_hyponyms_meronym(hypernymy_dict_sorted, hypernym_name, kind="meronyms")
        
        all_hypernyms.append(hypernym_name)
        all_hyponyms.append(hyponyms)
        all_meronyms.append(meronyms)
        all_hhm_map.append(hypernymy_dict_sorted)
    else:
        all_hypernyms.append("--NA--")
        all_hyponyms.append("--NA--")
        all_meronyms.append("--NA--")
        all_hhm_map.append(hypernymy_dict)

In [ ]:
extract_hyponyms_meronym(all_hhm_map[0], 'sanskrit', kind="hyponyms")

In [ ]:
all_meronyms

#### Hyponymy

In [ ]:
num_vals = len(all_hypernymy[2][all_hypernyms[2]]["rev_map"]["direct"])
hyponyms = []
meronyms = []
for i in range(0,num_vals):
    print(list(all_hypernymy[2][all_hypernyms[2]]["rev_map"]["direct"][i].values())[0].part_meronyms())

In [ ]:
num_vals2 = len(all_hypernymy[0][all_hypernyms[0]]["rev_map"]["l1"])
for i in range(0,num_vals2):
    print(list(all_hypernymy[0][all_hypernyms[0]]["rev_map"]["l1"][i].values())[0].part_meronyms())

#### Graph

In [2]:
hhm_mappings_file = os.path.join("hhm_mappings.csv")
sense_group_dataframe = pd.read_csv(hhm_mappings_file)

In [3]:
word = "day"

subset_df = sense_group_dataframe[sense_group_dataframe["sense_group_name"]==word]

In [4]:
subset_df

,Unnamed: 0,word,context,sense_id,sense_group_name,sense_group_num,sense_probability,related_senses,hhm_map,hypernym,hyponym,meronym,hypernym_from_corpus,hyponym_from_corpus,meronym_from_corpus
29,29,<day>,this is a big <day> last day Ill ever put on m...,day#3,day,3,0.999606,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
30,30,<day>,this is a big day last <day> Ill ever put on m...,day#3,day,3,0.999606,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
32,32,<day>,this is a big day last day Ill ever put on my ...,day#3,day,3,0.737632,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
34,34,<day>,last day Ill ever put on my leotard last day o...,day#3,day,3,0.737632,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
196,196,<day>,I ve been in bed all <day>\n,day#3,day,3,0.972958,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
322,322,<day>,happy fathers <day> to mr chalkley I was gon n...,day#3,day,3,0.662118,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
334,334,<day>,with do I actually have to do I d be here all ...,day#3,day,3,0.795181,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
363,363,<day>,I suppose I better do something with my <day> ...,day#3,day,3,0.891535,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
449,449,<day>,one <day> in uni another long weekend\n,day#3,day,3,0.941290,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"
455,455,<day>,wicked sticks what <day> are you movingSmile\n,day#3,day,3,0.712711,"['DAY#3', 'SESSION#4', 'Session#4', 'session#5...","{'time_period': defaultdict(<class 'dict'>, {'...",time_period,"['week_from_monday', 'wedding_night', 'lunar_d...","['late-night_hour', 'midweek', 'day_of_the_wee...",hour,"['evening', 'rag', 'night']","['weekend', 'evening', 'morning']"


In [5]:
unq_hypernyms = list(set(subset_df["hypernym"].values))
unq_hypernyms_from_cropus = list(set(subset_df["hypernym_from_corpus"].values))

In [49]:
hypo_mero_map = {}
unq_hyponyms_from_corpus = []
unq_meronyms_from_corpus = []
hyper_node_sizes = len(unq_hypernyms_from_cropus)
hypo_node_sizes = []
mero_node_sizes = []
for hyp in unq_hypernyms_from_cropus:
    hyper_df = subset_df[subset_df["hypernym_from_corpus"]==hyp]
    hyponyms = []
    meronyms = []
    for _, row in hyper_df[["hyponym_from_corpus", "meronym_from_corpus"]].iterrows():
        hypo, mero = row
        hyponyms += ast.literal_eval(hypo)
        meronyms += ast.literal_eval(mero)
    unq_hyponyms_from_corpus += hyponyms
    unq_meronyms_from_corpus += meronyms
    
    hypo_node_sizes.append(len(list(set(hyponyms))))
    mero_node_sizes.append(len(list(set(meronyms))))
    hypo_mero_map[hyp] = {
        "hyponyms": list(set(hyponyms)),
        "meronyms": list(set(meronyms))
    }

unq_hyponyms_from_corpus = list(set(unq_hyponyms_from_corpus))
unq_meronyms_from_corpus = list(set(unq_meronyms_from_corpus))


In [50]:
hypo_mero_map

{'eat': {'hyponyms': ['evening'],
  'meronyms': ['sunday', 'morning', 'saturday']},
 'hour': {'hyponyms': ['night', 'rag', 'evening'],
  'meronyms': ['weekend', 'morning', 'evening']}}

In [54]:
mero_node_sizes

[3, 3]

In [56]:
max(hyper_node_sizes, sum(hypo_node_sizes))

4

In [41]:
max_nodes

72

In [8]:
unq_meronyms_from_corpus

['sunday', 'saturday', 'morning', 'weekend', 'evening']

In [9]:
unq_hyponyms_from_corpus

['night', 'rag', 'evening']

In [116]:
import plotly.graph_objects as go
import igraph
from igraph import Graph, EdgeSeq

In [117]:
G = Graph()
G.add_vertex(word)

igraph.Vertex(<igraph.Graph object at 0x7f8f747236d8>, 0, {'name': 'day'})

In [118]:
vertices = list(set(unq_hyponyms_from_corpus+unq_hypernyms_from_cropus+unq_meronyms_from_corpus))
for n in vertices:
    print(n)
    G.add_vertex(n)
# G.add_vertex("pqr")
# G.add_vertex("xyz")
# G.add_vertex("mno")


night
sunday
rag
saturday
eat
morning
weekend
evening
hour


In [119]:
for h in unq_hypernyms_from_cropus:
    print(f"Adding edge - {word}-{h}")
    G.add_edge(word, h)

Adding edge - day-eat
Adding edge - day-hour


In [120]:
for hyper, mapping in hypo_mero_map.items():
    for hypo in mapping["hyponyms"]:
        print(f"Adding edges : {hyper}-{hypo}")
        G.add_edge(hyper, hypo)
    for mero in mapping["meronyms"]:
        print(f"Adding edges : {hyper}-{mero}")
        G.add_edge(hyper, mero)

Adding edges : eat-evening
Adding edges : eat-sunday
Adding edges : eat-morning
Adding edges : eat-saturday
Adding edges : hour-night
Adding edges : hour-rag
Adding edges : hour-evening
Adding edges : hour-weekend
Adding edges : hour-morning
Adding edges : hour-evening


In [121]:
# G.add_edges([("abc", "pqr"), ("pqr", "mno"), ("mno", "xyz"), ("xyz", "pqr")])

In [122]:

# nr_vertices = 25
# v_label = list(map(str, range(nr_vertices)))
# G = Graph.Tree(nr_vertices, 2) # 2 stands for children number
lay = G.layout('auto')

In [133]:
nr_vertices = 1 + len(vertices)
v_label = [word] + vertices
position = {k: lay[k] for k in range(nr_vertices)}
Y = [lay[k][1] for k in range(nr_vertices)]
M = max(Y)

es = EdgeSeq(G) # sequence of edges
E = [e.tuple for e in G.es] # list of edges

L = len(position)
Xn = [position[k][0] for k in range(L)]
Yn = [position[k][1] for k in range(L)]
# Yn = [2*M-position[k][1] for k in range(L)]
Xe = []
Ye = []
for edge in E:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
#     Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]
    Ye+=[position[edge[0]][1],position[edge[1]][1], None]

labels = v_label

In [134]:
position

{0: [0.13205307577144823, 0.07506480854265077],
 1: [1.204969293063682, -1.1472790075655315],
 2: [-0.3423112928170799, 1.5248673427059654],
 3: [1.5421541264193852, -0.27785656186951097],
 4: [-1.2440076411292238, 0.7334004719056273],
 5: [-0.3611737836099297, 0.6369845482077239],
 6: [-0.3198690865594072, -0.31802005673395584],
 7: [0.29916760808886905, -1.3688919774521184],
 8: [0.5804142850851812, 0.4722065692503996],
 9: [0.6340553088097759, -0.4968524719841706]}

In [135]:
E

[(0, 5),
 (0, 9),
 (5, 8),
 (2, 5),
 (5, 6),
 (4, 5),
 (1, 9),
 (3, 9),
 (8, 9),
 (7, 9),
 (6, 9),
 (8, 9)]

In [136]:
labels

['day',
 'night',
 'sunday',
 'rag',
 'saturday',
 'eat',
 'morning',
 'weekend',
 'evening',
 'hour']

In [137]:
Yn

[0.07506480854265077,
 -1.1472790075655315,
 1.5248673427059654,
 -0.27785656186951097,
 0.7334004719056273,
 0.6369845482077239,
 -0.31802005673395584,
 -1.3688919774521184,
 0.4722065692503996,
 -0.4968524719841706]

In [138]:
Xn

[0.13205307577144823,
 1.204969293063682,
 -0.3423112928170799,
 1.5421541264193852,
 -1.2440076411292238,
 -0.3611737836099297,
 -0.3198690865594072,
 0.29916760808886905,
 0.5804142850851812,
 0.6340553088097759]

In [139]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(y=Yn,
                  x=Xn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))
fig.show()

In [129]:
hypo_mero_map

{'eat': {'hyponyms': ['evening'],
  'meronyms': ['sunday', 'morning', 'saturday']},
 'hour': {'hyponyms': ['night', 'rag', 'evening'],
  'meronyms': ['weekend', 'morning', 'evening']}}

In [ ]:
def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
    L=len(pos)
    if len(text)!=L:
        raise ValueError('The lists pos and text must have the same len')
    annotations = []
    for k in range(L):
        annotations.append(
            dict(
                text=labels[k], # or replace labels with a different list for the text within the circle
                x=pos[k][0], y=2*M-position[k][1],
                xref='x1', yref='y1',
                font=dict(color=font_color, size=font_size),
                showarrow=False)
        )
    return annotations


In [ ]:
axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(title= 'Tree with Reingold-Tilford Layout',
              annotations=make_annotations(position, v_label),
              font_size=12,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=40, r=40, b=85, t=100),
              hovermode='closest',
              plot_bgcolor='rgb(248,248,248)'
              )
fig.show()

#### Custom Graph

In [254]:
import math

In [255]:
hypo_mero_map

{'eat': {'hyponyms': ['evening'],
  'meronyms': ['sunday', 'morning', 'saturday']},
 'hour': {'hyponyms': ['night', 'rag', 'evening'],
  'meronyms': ['weekend', 'morning', 'evening']}}

In [256]:
hyper_start = 0

In [257]:
word_y = 0
word_x = 0
hyper_y = 2
hyper_x = 0

hypo_y = -2

In [258]:
position = {}
edges = {}

In [259]:
hypo_mero_map["eat"]

{'hyponyms': ['evening'], 'meronyms': ['sunday', 'morning', 'saturday']}

In [260]:
hyper = "eat"
hypo = hypo_mero_map[hyper]['hyponyms']
mero = hypo_mero_map[hyper]['meronyms']

In [261]:
position[word] = [word_x, word_y]

In [262]:
position[hyper] = [hyper_x, hyper_y]
hyper_x +=2
hypo_start = 0 - math.ceil(len(hypo)/2) if len(hypo)>1 else 0
hypo_x = hypo_start
for h in hypo:
    if h in position:
        position[f"{h}_{hyper}"] = [hypo_x, hypo_y]
        hypo_x +=2
    else:
        position[h] = [hypo_x, hypo_y]
        hypo_x +=2

mero_start = 0 - math.ceil(len(mero)/2) if len(mero)>1 else 0
mero_x = mero_start    
for m in mero:
    if m in position:
        position[f"{m}_{hyper}"] = [mero_x, hypo_y-2]
        mero_x +=2
    else:
        position[m] = [mero_x, hypo_y-2]
        mero_x +=2


In [263]:
hyper = "hour"
hypo = hypo_mero_map[hyper]['hyponyms']
mero = hypo_mero_map[hyper]['meronyms']

In [264]:
position[hyper] = [hyper_x, hyper_y]
hyper_x +=2
hypo_start += 0 - math.ceil(len(hypo)/2) if len(hypo)>1 else 0
for h in hypo:
    if h in position:
        position[f"{h}_{hyper}"] = [hypo_x, hypo_y]
        hypo_x +=2
    else:
        position[h] = [hypo_x, hypo_y]
        hypo_x +=2

mero_start += 0 - math.ceil(len(mero)/2) if len(mero)>1 else 0  
for m in mero:
    if m in position:
        position[f"{m}_{hyper}"] = [mero_x, hypo_y-2]
        mero_x +=2
    else:
        position[m] = [mero_x, hypo_y-2]
        mero_x +=2


In [265]:
position

{'day': [0, 0],
 'eat': [0, 2],
 'evening': [0, -2],
 'sunday': [-2, -4],
 'morning': [0, -4],
 'saturday': [2, -4],
 'hour': [2, 2],
 'night': [2, -2],
 'rag': [4, -2],
 'evening_hour': [8, -4],
 'weekend': [4, -4],
 'morning_hour': [6, -4]}

In [266]:
labels = list(position.keys())
Xn = [position[k][0] for k in labels]
Yn = [position[k][1] for k in labels]


In [267]:

fig = go.Figure()
# fig.add_trace(go.Scatter(x=Xe,
#                    y=Ye,
#                    mode='lines',
#                    line=dict(color='rgb(210,210,210)', width=1),
#                    hoverinfo='none'
#                    ))
fig.add_trace(go.Scatter(y=Yn,
                  x=Xn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))
fig.show()

In [253]:
hypo_mero_map

{'eat': {'hyponyms': ['evening'],
  'meronyms': ['sunday', 'morning', 'saturday']},
 'hour': {'hyponyms': ['night', 'rag', 'evening'],
  'meronyms': ['weekend', 'morning', 'evening']}}